In [54]:
import pandas as pd
import numpy as np

data = pd.read_csv('WeatherData.csv')
data.iloc[:7]

,Unnamed: 0,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,0,2020-01-01,12:00 AM,81 F,73 F,79 %,N,5 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy
1,1,2020-01-01,12:30 AM,81 F,73 F,79 %,VAR,6 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy
2,2,2020-01-01,1:00 AM,79 F,73 F,83 %,N,5 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy
3,3,2020-01-01,1:30 AM,79 F,73 F,83 %,N,6 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy
4,4,2020-01-01,2:00 AM,79 F,73 F,83 %,N,6 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy
5,5,2020-01-01,2:30 AM,79 F,73 F,83 %,N,5 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy
6,6,2020-01-01,3:00 AM,79 F,73 F,83 %,N,3 mph,0 mph,29.89 in,0.0 in,Mostly Cloudy


In [55]:
data = data.drop(columns = ["Unnamed: 0", "Date", "Time", "Dew Point", "Wind", "Wind Speed", "Wind Gust", "Pressure", "Precip."])
data["Temperature"] = data["Temperature"].apply( lambda x: round((int(x[:2])-32) * 5/9 + 273, 1) )
data["Humidity"] = data["Humidity"].apply(lambda x: int(x[:2]))
labels = data.Condition.unique()
labels

array(['Mostly Cloudy', 'Fair', 'Partly Cloudy',
       'Showers in the Vicinity', 'Rain Shower', 'Light Rain Shower',
       'Light Rain', 'Rain', 'Heavy Rain', 'Haze',
       'Light Rain with Thunder', 'Heavy Rain Shower',
       'Thunder in the Vicinity', 'Heavy T-Storm', 'T-Storm', 'Thunder',
       'Fog', 'Mostly Cloudy / Windy', 'Cloudy', 'Light Rain / Windy',
       'Partly Cloudy / Windy'], dtype=object)

In [56]:
processed_data = data.copy()

norain = ['Mostly Cloudy', 'Fair', 'Partly Cloudy', 'Haze', 'Fog', 'Mostly Cloudy / Windy', 'Cloudy', 'Partly Cloudy / Windy']
mightrain = ['Showers in the Vicinity', 'Thunder in the Vicinity', 'Thunder']
rain = ['Rain Shower', 'Light Rain Shower', 'Light Rain', 'Rain', 'Heavy Rain', 'Light Rain with Thunder', 'Heavy Rain Shower', 'Heavy T-Storm', 'T-Storm', 'Light Rain / Windy']

processed_data["NoRain"] = 0
processed_data["MightRain"] = 0
processed_data["Rain"] = 0

processed_data["NoRain"].loc[data.Condition.isin(norain)] = 1
processed_data["MightRain"].loc[data.Condition.isin(mightrain)] = 1
processed_data["Rain"].loc[data.Condition.isin(rain)] = 1
processed_data = processed_data.drop(columns="Condition")

processed_data

,Temperature,Humidity,NoRain,MightRain,Rain
0,300.4,79,1,0,0
1,300.4,79,1,0,0
2,299.3,83,1,0,0
3,299.3,83,1,0,0
4,299.3,83,1,0,0
...,...,...,...,...,...
19160,299.3,83,1,0,0
19161,299.3,83,1,0,0
19162,298.1,89,1,0,0
19163,298.1,89,1,0,0


In [57]:
y = processed_data[["NoRain", "MightRain", "Rain"]].to_numpy()[7:]
rawx = processed_data.drop(columns = ["NoRain", "MightRain", "Rain"]).to_numpy()

x = []
temp = np.array(0)
for i in range(len(rawx)-7):
    temp = rawx[i:i+6].flatten()
    x.append(temp)
x = np.array(x)
x.shape, y.shape

((19158, 12), (19158, 3))

In [58]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Dense
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.20, random_state = 33)

In [59]:
model = tf.keras.Sequential()
model.add(Dense(14, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=10, validation_split=0.1)

test_results = model.evaluate(xtest, ytest, verbose=1)

model.save('model')

Epoch 1/10
432/432 [==============================] - 1s 880us/step - loss: 4.8984 - accuracy: 0.8484 - val_loss: 0.5925 - val_accuracy: 0.8663
Epoch 2/10
432/432 [==============================] - 0s 682us/step - loss: 0.6456 - accuracy: 0.8489 - val_loss: 0.5362 - val_accuracy: 0.8715
Epoch 3/10
432/432 [==============================] - 1s 1ms/step - loss: 0.5623 - accuracy: 0.8592 - val_loss: 0.5112 - val_accuracy: 0.8735
Epoch 4/10
432/432 [==============================] - 0s 885us/step - loss: 0.5731 - accuracy: 0.8567 - val_loss: 0.4927 - val_accuracy: 0.8728
Epoch 5/10
432/432 [==============================] - 0s 643us/step - loss: 0.5046 - accuracy: 0.8697 - val_loss: 1.0341 - val_accuracy: 0.2081
Epoch 6/10
432/432 [==============================] - 0s 620us/step - loss: 0.5388 - accuracy: 0.8453 - val_loss: 0.8484 - val_accuracy: 0.4318
Epoch 7/10
432/432 [==============================] - 0s 602us/step - loss: 0.5050 - accuracy: 0.8570 - val_loss: 0.4432 - val_accuracy: 0

In [60]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("model.tflite")
print("Model is %d bytes" % basic_model_size)

INFO:tensorflow:Assets written to: /var/folders/1z/7jpgpwcn0kzgx2q4k18ldvgr0000gn/T/tmpkylg5kbg/assets
INFO:tensorflow:Assets written to: /var/folders/1z/7jpgpwcn0kzgx2q4k18ldvgr0000gn/T/tmpkylg5kbg/assets
Model is 3156 bytes


In [61]:
!echo "const unsigned char model[] = {"  > model.h
!cat model.tflite | xxd -i              >> model.h
!echo "};"                              >> model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")

Header file, model.h, is 19,496 bytes.
